<a href="https://colab.research.google.com/github/Fangorne/Python_Notebook/blob/main/NLPSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [8]:
from transformers import AutoTokenizer, CamembertTokenizer, AutoModelForSequenceClassification, pipeline, AutoModelForTokenClassification
import torch
import torch.nn.functional as F

class SentimentDetection:

    def __init__(self, device):
        #  pretrained "nlptown/bert-base-multilingual-uncased-sentiment" or "cmarkea/distilcamembert-base-sentiment"
        self.model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name).to(device)
        # self.tokenizer = CamembertTokenizer.from_pretrained(self.model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.device = device


    def detectSentiment(model, text):

        tokens = model.tokenizer.tokenize(text)
        token_ids = model.tokenizer.convert_tokens_to_ids(tokens)
        input_ids = model.tokenizer(text)

        batch = model.tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt').to(model.device)
 
        with torch.no_grad():
            outputs = model.model(**batch)
            predictions = F.softmax(outputs.logits)
            labels = torch.argmax(predictions, dim=1)
            result = [model.model.config.id2label[label_id] for label_id in labels.tolist()]

        return result


    def save(model, save_directory):
        model.tokenizer.save_pretrained(save_directory)
        model.model.save_pretrained(save_directory)

sentimentDetection = SentimentDetection("cpu") 

print(sentimentDetection.detectSentiment("C'est vraiment génial"))

print(sentimentDetection.detectSentiment("C'est vraiment nul"))


['5 stars']
['1 star']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
